In [1]:
import os
import importlib
from datetime import datetime
import uuid

import db_connect
import bucket_connect
importlib.reload(db_connect)
importlib.reload(bucket_connect)

from path_management import PathManager
from bucket_connect import BucketConnector
from db_connect import DbConnector

In [2]:
scorecards_location = os.path.expanduser('~/Dropbox/Misc/Scorecards')

In [3]:
paths = PathManager([])
test_connector = DbConnector('./test.postgres_config', paths)
prod_connector = DbConnector('./prod.postgres_config', paths)
bucket = BucketConnector('./prod.bucket_config', paths)

In [8]:
connector = prod_connector
for file_name in os.listdir(scorecards_location):
    # find game
    game_name = file_name.removesuffix('.pdf')
    game_id = connector.get_game_by_name(game_name)
    if not game_id:
        try:
            game_date = datetime.strptime(game_name[:8], '%y-%m-%d')
            game_id = prod_connector.get_game_by_date(game_date)
        except Exception:
            print(f'failed to parse date from {file_name}')
    if not game_id:
        print(f'Could not identify unique game for {file_name}. Skipping')
        continue
    elif connector.has_scorecard(game_id):
        print(f'Game for {file_name} already has scorecard, skipping')
        continue
    else:
        print(f'Processing scorecard {file_name}')
    
    # upload file to bucket
    identifier = str(uuid.uuid4()).upper()
    extension = os.path.splitext(file_name)[1]
    key = f'{identifier}/original{extension}'
    if bucket.file_exists_by_key(key):
        print(f'scorecard already uploaded with identifier: {identifier}')
        continue
    file_path = os.path.join(scorecards_location, file_name)
    bucket.upload_by_key(file_path, key)

    # store record in db
    game = connector.get_game_by_id(game_id)
    connector.import_scorecard(game, identifier, file_name, extension)

Game for 19-05-27 Indians vs. Red Sox.pdf already has scorecard, skipping
Game for 14-09-23 Rays vs. Red Sox.pdf already has scorecard, skipping
Game for 21-09-22 Mets vs Red Sox.pdf already has scorecard, skipping
Game for 13-07-06 Pirates vs. Cubs.pdf already has scorecard, skipping
Game for 14-07-02 Cubs vs. Red Sox.pdf already has scorecard, skipping
Game for 12-07-15 Diamondbacks vs. Cubs.pdf already has scorecard, skipping
Game for 18-05-14 Braves vs. Cubs.pdf already has scorecard, skipping
Game for 13-09-17 Orioles vs. Red Sox.pdf already has scorecard, skipping
Game for 18-08-05 Yankees vs. Red Sox.pdf already has scorecard, skipping
Game for 21-10-05 ALWC Yankees vs Red Sox.pdf already has scorecard, skipping
Game for 14-08-24 Orioles vs. Cubs.pdf already has scorecard, skipping
Game for 17-10-11 NLDS Game 4 - Nationals vs. Cubs.pdf already has scorecard, skipping
Game for 14-05-30 Rays vs. Red Sox.pdf already has scorecard, skipping
Game for 12-08-12 Reds vs. Cubs.pdf alread